In [2]:
import pandas as pd
import numpy as np
import glob

In [7]:
cat = pd.read_csv('/Users/Nick/IS/Datasets/catalog_IMGs_USE.csv', delim_whitespace = True, skiprows = 0)

In [8]:
cat[0:5]

,ID,z_peak,UmV,VmJ,SFR_UV,SFR_IR,SFR_tot,LMASS,ra,dec,USE
0,34,0.3185,2.38,1.27,0.012,-0.794,0.012,9.71,150.38348,1.609724,0
1,60,0.3642,1.10,0.77,0.244,-1.134,0.244,9.17,150.40163,1.615784,1
2,85,0.3480,1.12,0.74,0.311,-1.004,0.311,9.24,150.37434,1.619130,1
3,136,0.3361,1.57,1.10,0.113,0.151,0.264,9.46,150.37727,1.627746,1
4,137,0.3600,1.22,0.79,0.682,0.310,0.992,9.68,150.39919,1.629076,1


In [7]:
ra = cat['ra']
dec = cat['dec']

In [67]:
bi = 4000
en = len(ra)
rd = open('/Users/Nick/IS/Text_Inputs/HST_RA_DEC_end', 'w')
rd.write("ra dec" + ' \n')
for ii in range (bi, en):
    rd.write(str(ra[ii]) + ' ' + str(dec[ii]) + '\n')
rd.close()

In [69]:
#below is all needed for determing what galaxies do not have a corrosponidng image cutout, due to server failure
#code can't be reused because all files have been renamed already - just here for reference
#split into 6 sections because NASA server can only pull max 800 images in a single request

In [16]:
filefits1 = glob.glob('/Users/Nick/IS/HST_Images/HST_0800/*.fits')
filefits2 = glob.glob('/Users/Nick/IS/HST_Images/HST_1600/*.fits')
filefits3 = glob.glob('/Users/Nick/IS/HST_Images/HST_2400/*.fits')
filefits4 = glob.glob('/Users/Nick/IS/HST_Images/HST_3200/*.fits')
filefits5 = glob.glob('/Users/Nick/IS/HST_Images/HST_4000/*.fits')
filefits6 = glob.glob('/Users/Nick/IS/HST_Images/HST_4265/*.fits')
print len(filefits1)
print len(filefits2)
print len(filefits3)
print len(filefits4)
print len(filefits5)
print len(filefits6)

775
744
778
576
766
199


In [17]:
filefits = filefits1 + filefits2 + filefits3 + filefits4 + filefits5 + filefits6
print len(filefits)

3838


In [8]:
zeroes = np.zeros(len(ra))
cat['HST_Image'] = zeroes

In [19]:
#Pulling Right Ascension and Declination information for all galaxies in Hubble dataset
ra_good = []
dec_good = []
for ii in range(0, len(filefits)):
    num = filefits[ii][40:60]
    nums = num.split('_')
    numsi = float(nums[0])
    dec = float(nums[1])
    ra_good.append(numsi)
    dec_good.append(dec)

In [21]:
#relating galaxies with images to their RA and DEC coordinates
for ii in range(0, len(ra_good)):
    col = cat[(cat['ra'] >= ra_good[ii]-.0001) & (cat['ra'] <= ra_good[ii]+.0001)& 
              (cat['dec'] >= dec_good[ii]-.0001) & (cat['dec'] <= dec_good[ii]+.0001)]
    index = col.index.values.tolist()
    index = int(index[0])
    cat.loc[index, 'HST_Image'] = 1

In [23]:
cat
cat.to_csv('/Users/Nick/IS/Datasets/catalog_IMGs1+HST_info', sep = ' ')

In [24]:
def zeropad(n,zeros=4):
    "Pad number n with zeros. Example: zeropad(7,3) == '007'"
    nstr = str(n)
    while len(nstr) < zeros:
        nstr = "0" + nstr
    return nstr

In [33]:
filefits = glob.glob('/Users/Nick/IS/HST_Images/All/*')
print len(filefits)

3838


In [34]:
rename = open('/Users/Nick/IS/Text_Inputs/rename2.txt', 'w')
rename.write("#!/bin/bash\n")

In [35]:
for ii in range(0, len(ra_good)):
    col = cat[(cat['ra'] >= ra_good[ii]-.0001) & (cat['ra'] <= ra_good[ii]+.0001)& 
              (cat['dec'] >= dec_good[ii]-.0001) & (cat['dec'] <= dec_good[ii]+.0001)]
    index = col.index.values.tolist()
    index = int(index[0])
    indexz = zeropad(index)
    ID = int(cat.loc[index, 'ID'])
    rename.write('mv '+ str(filefits[ii]) + ' /Users/Nick/IS/HST_Images/All/%sID%s.fits\n' % (str(indexz), str(ID)))

In [36]:
rename.close()


Below is for pulling all galaxies with an HST image and a USE value on the UVISTA dataset

In [11]:
cat_HST = pd.read_csv('/Users/Nick/IS/Datasets/catalog_IMGs1+HST_info', delim_whitespace = True)
USE = cat['USE']
cat_HST['USE'] = USE
cat_HST_o = cat_HST[(cat_HST['HST_Image'] == 1.0)]
cat_HST_o = cat_HST_o.reset_index()
del cat_HST_o['index']
USE2 = cat_HST_o['USE']
cat_HST2 = cat_HST[(cat_HST['HST_Image'] == 1.0) & (cat_HST['USE']== 1)]
#this is the catalog that contains all galaxys for which there is a HST image and USE = 1

In [12]:
cat_HST2 = cat_HST2.reset_index()
#index must be reset so there are no missing values
del cat_HST2['index']

In [102]:
cat_HST2.to_csv('/Users/Nick/IS/Datasets/cat_USE.csv', sep = ' ')